# Black-Scholes Option Pricing formula

In [4]:
import numpy as np
from scipy.stats import norm

In [48]:
def bs_optionpricing(S, K, sigma, r, T, delta, option_type="call"):
    
    assert option_type in ["call", "put"], "option_type only accepts 'call' or 'put' as argument"
    
    d1 = (np.log(S/K) + (r-delta+(sigma**2)/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    if option_type == "call":
        price = S*np.exp(-delta*T)*norm.cdf(d1,0,1) - K*np.exp(-r*T)*norm.cdf(d2,0,1)
        
        print("To dynamically replicate payoff of the call at expiration,")
        print("Delta: ", np.exp(-delta*T)*norm.cdf(d1,0,1))
        print("B:", -K*np.exp(r*T)*norm.cdf(d2,0,1))
        
    elif option_type == "put":
        price = K*np.exp(-r*T)*norm.cdf(-d2,0,1) - S*np.exp(-delta*T)*norm.cdf(-d1,0,1)
        
        print("To dynamically replicate payoff of the put at expiration,")
        print("Delta: ", -np.exp(-delta*T)*norm.cdf(d1,0,1))
        print("B:", K*np.exp(r*T)*norm.cdf(d2,0,1))

    return price

In [49]:
S = 52
K = 50
sigma = 0.18
r = 0.01
T = 6/12
delta = 0

call_price = bs_optionpricing(S, K, sigma, r, T, delta, option_type="call")
put_price = bs_optionpricing(S, K, sigma, r, T, delta, option_type="put")

print("Call price: ", call_price)
print("Put price: ", put_price)

To dynamically replicate payoff of the call at expiration,
Delta:  0.6594895230130656
B: -30.739050238164193
To dynamically replicate payoff of the put at expiration,
Delta:  -0.6594895230130656
B: 30.739050238164193
Call price:  3.8602636187776227
Put price:  1.6108875784117451


In [51]:
S = 53
K = 50
sigma = 0.18
r = 0.01
T = 25/52
delta = 0

call_price = bs_optionpricing(S, K, sigma, r, T, delta, option_type="call")
call_price

To dynamically replicate payoff of the call at expiration,
Delta:  0.7149128101480642
B: -33.71733789666272


4.495692518335147

Do you know that we can use the price of a put through the price of otherwise identical call option with put call parity?

In [16]:
def put_price_conversion(call_price, K, F, r, T): # F is the forward price of the underlying asset
    return call_price + K*np.exp(-r*T) - F # by put call parity, P = C + PV(K-F)

In [17]:
put_price_conversion(call_price, K, S, r, T)

1.6070251195071066

## Option Greeks with Black-Scholes

Reference: https://www.youtube.com/watch?v=558k7D2alxM

### Delta

Delta measures the rate of change of the theoretical option value with respect to changes in the underlying asset's price.

$\Delta = \frac{\partial V}{\partial S}$

$\Delta_{call} = \Phi(d1)$

$\Delta_{put} = -\Phi(-d1)$

In [18]:
def delta_calc(S, K, sigma, r, T, delta, option_type="call"):
    
    assert option_type in ["call", "put"], "option_type only accepts 'call' or 'put' as argument"

    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    if option_type == "call":
        delta_calc = norm.cdf(d1, 0, 1)
    elif option_type == "put":
        delta_calc = -norm.cdf(-d1, 0, 1)
    
    return delta_calc

### Gamma

Gamma measures the rate of change in the delta with respect to changes in the underlying price.

$\Gamma = \frac{\partial \Delta}{\partial S} = \frac{\partial^2 V}{\partial S^2}$

$\Gamma = \frac{\phi(d1)}{S\sigma\sqrt{\tau}}$

In [20]:
def gamma_calc(S, K, sigma, r, T, delta, option_type="call"):
    
    assert option_type in ["call", "put"], "option_type only accepts 'call' or 'put' as argument"
    
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    gamma_calc = norm.pdf(d1, 0, 1)/(S*sigma*np.sqrt(T))
    
    return gamma_calc

### Vega

Vega measures sensitivity to volatility. Vega is the derivative of the option value with respect to the volatility of the underlying asset.

$\upsilon = \frac{\partial V}{\partial \sigma}$

$\upsilon = S\phi(d1)\sqrt{\tau}$

In [19]:
def vega_calc(S, K, sigma, r, T, delta, option_type="call"):

    assert option_type in ["call", "put"], "option_type only accepts 'call' or 'put' as argument"

    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    vega_calc = S*norm.pdf(d1, 0, 1)*np.sqrt(T)
    
    return vega_calc*0.01

### Theta

Theta measures the sensitivity of the value of the derivative to the passage of time - time decay.

$\Theta = -\frac{\partial V}{\partial \tau}$

$\Theta_{call} = -\frac{S\phi(d1)\sigma}{2\tau} - rK\exp{(-rT)}\Phi(d2)$

$\Theta_{put} = -\frac{S\phi(d1)\sigma}{2\tau} + rK\exp{(-rT)}\Phi(-d2)$

In [21]:
def theta_calc(S, K, sigma, r, T, delta, option_type="call"):
    
    assert option_type in ["call", "put"], "option_type only accepts 'call' or 'put' as argument"

    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    if option_type == "call":
        theta_calc = -S*norm.pdf(d1, 0, 1)*sigma/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*norm.cdf(d2, 0, 1)
    elif option_type == "put":
        theta_calc = -S*norm.pdf(d1, 0, 1)*sigma/(2*np.sqrt(T)) + r*K*np.exp(-r*T)*norm.cdf(-d2, 0, 1)
        
    return theta_calc/365, theta(type, S, K, T, r, sigma)

### Rho

Rho measures the sensitivity to the interest rate.

$\rho = \frac{\partial V}{\partial r}$

$\rho_{call} = K\tau\exp{(-rT)}\Phi(d2)$

$\rho_{put} = -K\tau\exp{(-rT)}\Phi(-d2)$

In [22]:
def rho_calc(S, K, sigma, r, T, delta, option_type="call"):
    
    assert option_type in ["call", "put"], "option_type only accepts 'call' or 'put' as argument"
    
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    if type == "call":
        rho_calc = K*T*np.exp(-r*T)*norm.cdf(d2, 0, 1)
    elif type == "put":
        rho_calc = -K*T*np.exp(-r*T)*norm.cdf(-d2, 0, 1)
    
    return rho_calc*0.01, rho(type, S, K, T, r, sigma)

# Practical

> Noted that py_vollib black scholes model does not take into account the continuously compounded dividend yield 

In [ ]:
!pip install py_vollib
from py_vollib.black_scholes import black_scholes as bs
from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega, theta, rho

In [24]:
bs('c', S, K, T, r, sigma)

3.3990781872368983

In [29]:
delta('c', S, K, T, r, sigma)

0.6454074505086169

In [30]:
gamma('c', S, K, T, r, sigma)

0.060510598576190594

In [31]:
vega('c', S, K, T, r, sigma)

0.07628873715493228

In [32]:
theta('c', S, K, T, r, sigma)

-0.017595436745165064

In [33]:
rho('c', S, K, T, r, sigma)

0.05765656820904099